In [1]:
# Imports 
import string
import pandas as pd
import numpy as np
import time
from scipy.stats.stats import pearsonr 
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.neighbors import NearestNeighbors
import operator
from scipy.sparse import csr_matrix

In [2]:
# load the data, keep only selected columns
# Read the file using super fast Pandas.read_csv
def load_required_data(path, required_columns):
    dataframe = pd.read_json(path, lines=True)
    dataframe = dataframe[required_columns]
    return dataframe, dataframe.values

In [3]:
dataframe, values = load_required_data('../../Datasets/Apps Dataset/train_15000.json', ["asin", "reviewerID", "overall"])

In [4]:
dataframe[:10]

,asin,reviewerID,overall
0,B00ANT8OF6,A00100742Q4O8VH0YMUBZ,4
1,B00AMR1HZ8,A00100742Q4O8VH0YMUBZ,4
2,B004WGGQPQ,A00100742Q4O8VH0YMUBZ,5
3,B006C1ZSO4,A00100742Q4O8VH0YMUBZ,5
4,B008LY1B32,A00100742Q4O8VH0YMUBZ,4
5,B007KPT2N4,A00100742Q4O8VH0YMUBZ,1
6,B0080JJLBW,A00100742Q4O8VH0YMUBZ,5
7,B009P8EMCK,A00100742Q4O8VH0YMUBZ,5
8,B00I8Q77Y0,A001619027H9L9EG4UVRB,5
9,B00H0BGCJK,A001619027H9L9EG4UVRB,5


In [5]:
dataframe.shape

(15000, 3)

In [6]:
# Create user-item matrix given a matrix and mapping from original matrix columns to new matrix rows/columns
def create_user_item_matrix(data, rowMapping, columnMapping):
    # Create table of users to items by taking row and column pivots
    # Rows of user-item table are users
    rows, row_pos = np.unique(data[:, rowMapping], return_inverse=True)
    # Columns of user-item table are movies
    cols, col_pos = np.unique(data[:, columnMapping], return_inverse=True)

    pivot_table = np.zeros((len(rows), len(cols)), dtype=data.dtype)
    pivot_table[row_pos, col_pos] = data[:, 2]
    return pivot_table, rows, cols

In [7]:
ratings_matrix, rows, cols = create_user_item_matrix(values, rowMapping=1, columnMapping=0)
print (ratings_matrix.shape)

(3139L, 1842L)


In [8]:
pd.DataFrame(ratings_matrix[:10], index = rows[:10], columns= cols)

,B004ALVL6W,B004ANMWPY,B004DLNC4I,B004DLPXAO,B004DM1OAQ,B004DM1ZQY,B004DPBGCO,B004DPC5Y2,B004DPCSKI,B004DPIEF6,...,B00K7WGUKA,B00KFNXUY0,B00KGCNRAM,B00KI5Q8X0,B00KMX5V8G,B00KOEHQCW,B00KQHVWWC,B00KSOQ66K,B00KWVZ750,B00L3MNCNQ
A00100742Q4O8VH0YMUBZ,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A001619027H9L9EG4UVRB,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A002359833QJM7OQHCXWY,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A0037670NPLI11RBWYFA,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A003841815JTX0JFLR8B1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A0038872349TB5N0JHQQW,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A00455683H6M1GQZMPQPV,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,5,0
A0047670XAAJD587LXS7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A00501041JRAPWYLPQ4TE,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A00507662MEMHI1YMGQ15,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
def get_index_map(arr):
    index_map = {}
    for el in arr:
        index_map[el] = len(index_map)
    return index_map

In [10]:
def pearson_coeff(target, others):
    return np.corrcoef(matrix[target], matrix[others])[0][1:]

In [11]:
# Get unique movies and users
unique_items = np.unique(values[:,0])
unique_users = np.unique(values[:,1])

In [12]:
unique_items.shape

(1842L,)

In [13]:
unique_users.shape

(3139L,)

In [14]:
# Create mapping from unique list to 
item_idx_map = get_index_map(unique_items)
user_idx_map = get_index_map(unique_users)

In [15]:
mean_ratings = np.true_divide(ratings_matrix.sum(1), (ratings_matrix!=0).sum(1))

In [16]:
K = 0.1

In [17]:
def get_users_who_rated_item(item_id):
    return np.where(ratings_matrix[:,item_idx_map[item_id]] > 0)[0]

def get_users_who_rated_item_by_index(item_idx):
    return np.where(ratings_matrix[:,item_idx] > 0)

def predict(user_id, item_id):
    item_idx = item_idx_map[item_id]
    user_idx = user_idx_map[user_id]
    other_users_idx = get_users_who_rated_item(item_id)
    if user_idx in other_users_idx:
        np.delete(other_users_idx, user_idx)
    pearson_correlation = [pearsonr(other_user, ratings_matrix[user_idx])[0] for other_user in ratings_matrix[other_users_idx]]
    mean_others = mean_ratings[other_users_idx]
    rating_others = ratings_matrix[other_users_idx,item_idx]
    prediction = mean_ratings[user_idx] + K * np.sum(pearson_correlation * (rating_others - mean_others))
    return prediction

In [18]:
np.mean(mean_ratings)

4.100578451642274

In [19]:
test_dataframe, test_values = load_required_data('../../Datasets/Apps Dataset/test_5000.json', ["asin", "reviewerID", "overall"])

In [20]:
test_dataframe['reviewerID'].unique().shape

(2507L,)

In [21]:
# Returns DF containing predictions, MAE, RMSE
def get_predictions(dataframe):
    df = dataframe.copy()
    print ("Predicting ratings..")
    start_time = time.time()
    df['Prediction'] = df.apply(lambda row: predict_row(row), axis=1)
    MAE = round(mean_absolute_error(df['overall'], df['Prediction']), 2)
    RMSE = round(np.sqrt(mean_squared_error(df['overall'], df['Prediction'])), 2)
    print "Done Predicting in",  round((time.time() - start_time), 2), "seconds"
    return df, MAE, RMSE

def predict_row(row):
    prediction = predict(row['reviewerID'], row['asin'])
    print ("Actual = ", row['overall'], "Predictioned = ", prediction)
    return prediction

In [22]:
sparse_ratings = csr_matrix(ratings_matrix, dtype='float')

In [23]:
sparse_ratings

<3139x1842 sparse matrix of type '<type 'numpy.float64'>'
	with 15000 stored elements in Compressed Sparse Row format>

In [24]:
model_knn = NearestNeighbors(metric = 'cosine', n_neighbors = 6)
model_knn.fit(sparse_ratings)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='cosine',
         metric_params=None, n_jobs=1, n_neighbors=6, p=2, radius=1.0)

In [25]:
def get_recommendations(user_index, n_reccomendations=5, n_neighbors=10):
    distances,indices = model_knn.kneighbors(ratings_matrix[user_index].reshape(1, -1), n_neighbors=n_neighbors)
    probable_items = list()
    for user in indices[0]:
        if user == user_index:
            print "Recommendations for user ", unique_users[user_index]
            continue
        # all other users
        for items in np.nonzero(ratings_matrix[user]):
            for item in items:
                if ratings_matrix[user_index][item] == 0:
                    probable_items.append(unique_items[item])

    predictions = {}
    for item in probable_items:
        predictions[item] = predict(unique_users[0], item)
        sorted_predictions = sorted(predictions.items(), key=operator.itemgetter(1))
    print ("Choosing 5 out of ", len(sorted_predictions))
    return [i[0] for i in sorted_predictions[:n_reccomendations]]

In [26]:
for i in get_recommendations(user_index=1):
    print i

Recommendations for user  A001619027H9L9EG4UVRB
('Choosing 5 out of ', 9)
B00FAPF5U0
B009UX2YAC
B00A6S051A
B00AIUUXHC
B00992CF6W


In [27]:
def get_user_items_rated_df(user_id):
    df = pd.DataFrame()
    np.nonzero(ratings_matrix[user_id])
    for i in np.nonzero(ratings_matrix[user_id]):
        x = pd.DataFrame({"item":unique_items[i], "rating":ratings_matrix[user_id][i]})
        df = df.append(x)
        return df

def print_items_rated(user_id):
    np.nonzero(ratings_matrix[user_id])
    for i in unique_items[np.nonzero(ratings_matrix[user_id])]:
        print i

In [28]:
get_user_items_rated_df(0)

,item,rating
0,B004WGGQPQ,5
1,B006C1ZSO4,5
2,B007KPT2N4,1
3,B0080JJLBW,5
4,B008LY1B32,4
5,B009P8EMCK,5
6,B00AMR1HZ8,4
7,B00ANT8OF6,4


In [29]:
print_items_rated(5)

B00846GKTW
B008G0HTUU
B009GXHOW4
B009NOQG9S
B00AO2P3GA
B00APKHA9Y
B00DYOOGYK
B00GTCECVG
